In [1]:
import random

# Encoding

In [2]:
Q = 41

In [3]:
def encode(x):
    return x % Q

def decode(x):
    return x if x <= Q/2 else x-Q

In [4]:
x = encode(-5)
print("encoded: %d" % x)
print("decoded: %d" % decode(x))

encoded: 36
decoded: -5


# Additive sharing

In [5]:
N = 10

In [6]:
def additive_share(secret):
    shares  = [ random.randrange(Q) for _ in range(N-1) ]
    shares += [ (secret - sum(shares)) % Q ]
    return shares

def additive_reconstruct(shares):
    return sum(shares) % Q

shares = additive_share(5)
print(shares)
print(additive_reconstruct(shares))

[17, 2, 15, 25, 0, 19, 3, 37, 20, 31]
5


In [7]:
seen_shares = shares[:N-1]

for guess in range(Q):
    for unseen_share in range(Q):
        simulated_shares = seen_shares + [unseen_share]
        if additive_reconstruct(simulated_shares) == guess:
            print("guess %d explained by %d" % (guess, unseen_share))
            break
    else:
        print("guess %d could not be explained" % guess)

guess 0 explained by 26
guess 1 explained by 27
guess 2 explained by 28
guess 3 explained by 29
guess 4 explained by 30
guess 5 explained by 31
guess 6 explained by 32
guess 7 explained by 33
guess 8 explained by 34
guess 9 explained by 35
guess 10 explained by 36
guess 11 explained by 37
guess 12 explained by 38
guess 13 explained by 39
guess 14 explained by 40
guess 15 explained by 0
guess 16 explained by 1
guess 17 explained by 2
guess 18 explained by 3
guess 19 explained by 4
guess 20 explained by 5
guess 21 explained by 6
guess 22 explained by 7
guess 23 explained by 8
guess 24 explained by 9
guess 25 explained by 10
guess 26 explained by 11
guess 27 explained by 12
guess 28 explained by 13
guess 29 explained by 14
guess 30 explained by 15
guess 31 explained by 16
guess 32 explained by 17
guess 33 explained by 18
guess 34 explained by 19
guess 35 explained by 20
guess 36 explained by 21
guess 37 explained by 22
guess 38 explained by 23
guess 39 explained by 24
guess 40 explained b

In [8]:
def additive_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def additive_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [9]:
class Additive:
    
    def __init__(self, secret=None):
        self.shares = additive_share(encode(secret)) if secret is not None else []
    
    def reveal(self):
        return decode(additive_reconstruct(self.shares))
    
    def __repr__(self):
        return "Additive(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Additive()
        z.shares = additive_add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = Additive()
        z.shares = additive_sub(x.shares, y.shares)
        return z

x = Additive(5)
print(x)

y = Additive(8)
print(y)

z = x - y
print(z)
assert(z.reveal() == -3)

Additive(5)
Additive(8)
Additive(-3)


# Coefficient polynomials

In [10]:
# using Horner's rule

def evaluate_polynomial_at_point(coefs, point):
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % Q
    return result

In [11]:
class CoefficientPolynomial:
    
    def __init__(self, coefficients):
        self.coefs = coefficients
        
    def __repr__(self):
        return "CoefficientPolynomial(%s)" % self.coefs
        
    def value_at(self, point):
        return evaluate_polynomial_at_point(self.coefs, point)
    
    def values_at(self, points):
        return [ self.value_at(p) for p in points ]

poly = CoefficientPolynomial([5,6,7])
print(poly)
assert(poly.values_at([0,1,2]) == [(5 + 6*0 + 7*0) % Q, (5 + 6*1 + 7*1*1) % Q, (5 + 6*2 + 7*2*2) % Q])
print(poly.values_at([9,8,7]))

CoefficientPolynomial([5, 6, 7])
[11, 9, 21]


# Point-Value polynomials

In [12]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# from http://www.ucl.ac.uk/~ucahcjm/combopt/ext_gcd_python_programs.pdf
def egcd_binary(a,b):
    u, v, s, t, r = 1, 0, 0, 1, 0
    while (a % 2 == 0) and (b % 2 == 0):
        a, b, r = a//2, b//2, r+1
    alpha, beta = a, b
    while (a % 2 == 0):
        a = a//2
        if (u % 2 == 0) and (v % 2 == 0):
            u, v = u//2, v//2
        else:
            u, v = (u + beta)//2, (v - alpha)//2
    while a != b:
        if (b % 2 == 0):
            b = b//2
            if (s % 2 == 0) and (t % 2 == 0):
                s, t = s//2, t//2
            else:
                s, t = (s + beta)//2, (t - alpha)//2
        elif b < a:
            a, b, u, v, s, t = b, a, s, t, u, v
        else:
            b, s, t = b - a, s - u, t - v
    return (2 ** r) * a, s, t


def inverse(a):
    _, b, _ = egcd_binary(a, Q)
    return b

In [13]:
# see https://en.wikipedia.org/wiki/Lagrange_polynomial

def lagrange_constants_for_point(points, point):
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - point)) % Q
                denum = (denum * (xj - xi)) % Q
        constants[i] = (num * inverse(denum)) % Q
    return constants

def interpolate_polynomial_at_point(points_values, point):
    points, values = zip(*points_values)
    constants = lagrange_constants_for_point(points, point)
    return sum( vi * ci for vi, ci in zip(values, constants) ) % Q

In [14]:
class PointValuePolynomial:
    
    def __init__(self, points_values):
        self.pvs = points_values
        
    def __repr__(self):
        return "PointValuePolynomial(%s)" % self.pvs
    
    def value_at(self, point):
        return interpolate_polynomial_at_point(self.pvs, point)
    
    def values_at(self, points):
        return [ self.value_at(p) for p in points ]

poly = PointValuePolynomial([(9,11), (8,9), (7,21)])
print(poly)
print(poly.value_at(0))

PointValuePolynomial([(9, 11), (8, 9), (7, 21)])
5


# Shamir sharing

In [15]:
N = 10
T = 4

assert(T+1 <= N)

In [16]:
def sample_shamir_polynomial(zero_value):
    coefs = [zero_value] + [random.randrange(Q) for _ in range(T)]
    return CoefficientPolynomial(coefs)

In [17]:
SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(0 not in SHARE_POINTS)

def shamir_share(secret):
    poly = sample_shamir_polynomial(secret)
    return poly.values_at(SHARE_POINTS)

def shamir_reconstruct(shares):
    points_values = [ (p,v) for p,v in zip(SHARE_POINTS, shares) if v is not None ]
    print(points_values)
    poly = PointValuePolynomial(points_values)
    return poly.value_at(0)

shares = shamir_share(5)
#for i in range(N-(T+1)):
#    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
shares[0] = 4
x = shamir_reconstruct(shares)
print(x)
assert(x == 5)

[(1, 4), (2, 19), (3, 5), (4, 16), (5, 32), (6, 10), (7, 7), (8, 16), (9, 7), (10, 9)]
28


AssertionError: 

In [18]:
#pv = [(3, 29), (4, 18), (5, 19), (6, 1), (7, 24), (8, 34)]
pv = [(3, 29), (4, 18), (5, 19), (6, 1), (7, 24), (9, 27)]
poly = PointValuePoly(pv)
poly.value_at(0)

NameError: name 'PointValuePoly' is not defined

In [19]:
def shamir_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def shamir_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
def shamir_mul(x, y):
    return [ (xi * yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
class Shamir:
    
    def __init__(self, secret=None):
        self.shares = shamir_share(encode(secret)) if secret is not None else []
        self.degree = T
    
    def reveal(self):
        assert(self.degree+1 <= N)
        return decode(shamir_reconstruct(self.shares))
    
    def __repr__(self):
        return "Shamir(%d)" % self.reveal()
    
    def __add__(x, y):
        z = Shamir()
        z.shares = shamir_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Shamir()
        z.shares = shamir_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Shamir()
        z.shares = shamir_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z
    
x = Shamir(2)
print(x)

y = Shamir(3)
print(y)

z = x - y
print(z)
assert(z.reveal() == -1)

v = x * y
print(v)
assert(v.reveal() == 6)

# Packed sharing

In [ ]:
N = 20
T = 8
K = 2

assert(T+K <= N)

In [ ]:
SECRET_POINTS =     [ -p % Q for p in range(1, K+1) ]
RANDOMNESS_POINTS = [ -p % Q for p in range(K+1, K+T+1) ]
assert(set(SECRET_POINTS).intersection(RANDOMNESS_POINTS) == set())

def sample_packed_polynomial(secrets):
    assert(len(secrets) == K)
    points = SECRET_POINTS + RANDOMNESS_POINTS
    values = secrets + [ random.randrange(Q) for _ in range(T) ]
    return list(zip(points, values))

In [ ]:
SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(set(SHARE_POINTS).intersection(SECRET_POINTS) == set())
assert(set(SHARE_POINTS).intersection(RANDOMNESS_POINTS) == set())

def packed_share(secrets):
    poly = sample_packed_polynomial(secrets)
    return [ interpolate_polynomial_at_point(poly, p) for p in SHARE_POINTS ]

def packed_reconstruct(shares):
    points = SHARE_POINTS
    values = shares
    points_values = [ (p,v) for p,v in zip(points, values) if v is not None ]
    return [ interpolate_polynomial_at_point(points_values, p) for p in SECRET_POINTS ]

secrets = [5,6]
shares = packed_share(secrets)
for i in range(N-(T+K)):
    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
reconstructed_secrets = packed_reconstruct(shares)
assert(reconstructed_secrets == secrets)

In [ ]:
def packed_add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def packed_sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
def packed_mul(x, y):
    return [ (xi * yi) % Q for xi, yi in zip(x, y) ]

In [ ]:
class Packed:
    
    def __init__(self, secrets=None):
        self.shares = packed_share([ encode(s) for s in secrets ]) if secrets is not None else []
        self.degree = T+K-1
    
    def reveal(self):
        assert(self.degree+1 <= N)
        #print(packed_reconstruct(self.shares))
        return [ decode(s) for s in packed_reconstruct(self.shares) ]
    
    def __repr__(self):
        return "Packed(%s)" % self.reveal()
    
    def __add__(x, y):
        z = Packed()
        z.shares = packed_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Packed()
        z.shares = packed_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Packed()
        z.shares = packed_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z
    
x = Packed([2,3])
print(x)

y = Packed([2,3])
print(y)

z = x - y
print(z)
assert(z.reveal() == [0,0])

v = x * y
print(v)
assert(v.reveal() == [4,9])

# Fast Fourier Transform

In [20]:
N = 8
T = 4
K = 3
Q = 433 # 6 * (T+K+1) * (N+1)

assert(T+K+1 == 2**3)
assert(N+1 == 3**2)

In [21]:
OMEGA2 = 354
assert((OMEGA2**(T+K+1)) % Q == 1)

OMEGA3 = 150
assert((OMEGA3**(N+1)) % Q == 1)

points2 = set( (OMEGA2 ** e) % Q for e in range(Q) )
points3 = set( (OMEGA3 ** e) % Q for e in range(Q) )
print("Points used for sampling: %s" % sorted(points2))
print("Points used for sharing:  %s" % sorted(points3))
assert(points2.intersection(points3) == set([1]))

Points used for sampling: [1, 79, 148, 179, 254, 285, 354, 432]
Points used for sharing:  [1, 27, 150, 153, 198, 234, 256, 296, 417]


In [26]:
OMEGA4 = 354 ** 2 % Q
assert((OMEGA4**((T+K)//2+1)) % Q == 1)

points4 = set( (OMEGA4 ** e) % Q for e in range(Q) )
print("Points used for sampling: %s" % sorted(points4))

Points used for sampling: [1, 179, 254, 432]


In [26]:
assert((OMEGA2**((T+K)//2+1)) % Q == 1)

points4 = set( (OMEGA4 ** e) % Q for e in range(Q) )
print("Points used for sampling: %s" % sorted(points4))

Points used for sampling: [1, 179, 254, 432]


In [ ]:
# len(aX) must be a power of 2
def fft2_forward(aX, omega=OMEGA2):
    if len(aX) == 1:
        return aX

    # split A(x) into B(x) and C(x) -- A(x) = B(x^2) + x C(x^2)
    bX = aX[0::2]
    cX = aX[1::2]
    
    # apply recursively
    omega_squared = (omega**2) % Q
    B = fft2_forward(bX, omega_squared)
    C = fft2_forward(cX, omega_squared)
        
    # combine subresults
    A = [0] * len(aX)
    Nhalf = len(aX) // 2
    for i in range(0, Nhalf):
        
        j = i
        x = (omega**j) % Q
        A[j] = (B[i] + x * C[i]) % Q
        
        j = i + Nhalf
        x = (omega**j) % Q
        A[j] = (B[i] + x * C[i]) % Q
        
    return A

def fft2_backward(A):
    N_inv = inverse(len(A))
    return [ (a * N_inv) % Q for a in fft2_forward(A, inverse(OMEGA2)) ]

coefs = [1,2,3,4,5,6,7,8]
values = fft2_forward(coefs)
coefs_recovered = fft2_backward(values)
assert(coefs == coefs_recovered)

In [ ]:
# len(aX) must be a power of 3
def fft3_forward(aX, omega=OMEGA3):
    if len(aX) == 1:
        return aX

    # split A(x) into B(x), C(x), and D(x): A(x) = B(x^3) + x C(x^3) + x^2 D(x^3)
    bX = aX[0::3]
    cX = aX[1::3]
    dX = aX[2::3]
    
    # apply recursively
    omega_cubed = (omega**3) % Q
    B = fft3_forward(bX, omega_cubed)
    C = fft3_forward(cX, omega_cubed)
    D = fft3_forward(dX, omega_cubed)
        
    # combine subresults
    A = [0] * len(aX)
    Nthird = len(aX) // 3
    for i in range(Nthird):
        
        j = i
        x = (omega**j) % Q
        xx = (x * x) % Q
        A[j] = (B[i] + x * C[i] + xx * D[i]) % Q
        
        j = i + Nthird
        x = (omega**j) % Q
        xx = (x * x) % Q
        A[j] = (B[i] + x * C[i] + xx * D[i]) % Q
        
        j = i + Nthird + Nthird
        x = (omega**j) % Q
        xx = (x * x) % Q
        A[j] = (B[i] + x * C[i] + xx * D[i]) % Q

    return A

def fft3_backward(A):
    N_inv = inverse(len(A))
    return [ (a * N_inv) % Q for a in fft3_forward(A, inverse(OMEGA3)) ]

coefs = [1,2,3,4,5,6,7,8,9]
values = fft3_forward(coefs)
coefs_recovered = fft3_backward(values)
assert(coefs == coefs_recovered)

## Slightly optimised

In [ ]:
# len(aX) must be a power of 2
def fft2_forward(aX, omega=OMEGA2):
    if len(aX) == 1:
        return aX

    # split A(x) into B(x) and C(x) -- A(x) = B(x^2) + x C(x^2)
    bX = aX[0::2]
    cX = aX[1::2]
    
    # apply recursively
    omega_squared = (omega**2) % Q
    B = fft2_forward(bX, omega_squared)
    C = fft2_forward(cX, omega_squared)
        
    # combine subresults
    A = [0] * len(aX)
    Nhalf = len(aX) >> 1
    for i in range(0, Nhalf):
        
        x = (omega**i) % Q
        A[i]         = (B[i] + x * C[i]) % Q
        A[i + Nhalf] = (B[i] - x * C[i]) % Q
        
    return A

def fft2_backward(A):
    N_inv = inverse(len(A))
    return [ (a * N_inv) % Q for a in fft2_forward(A, inverse(OMEGA2)) ]

coefs = [1,2,3,4,5,6,7,8]
values = fft2_forward(coefs)
coefs_recovered = fft2_backward(values)
assert(coefs == coefs_recovered)

In [ ]:
# len(aX) must be a power of 3
def fft3_forward(aX, omega=OMEGA3):
    if len(aX) == 1:
        return aX

    # split A(x) into B(x), C(x), and D(x): A(x) = B(x^3) + x C(x^3) + x^2 D(x^3)
    bX = aX[0::3]
    cX = aX[1::3]
    dX = aX[2::3]
    
    # apply recursively
    omega_cubed = (omega**3) % Q
    B = fft3_forward(bX, omega_cubed)
    C = fft3_forward(cX, omega_cubed)
    D = fft3_forward(dX, omega_cubed)
        
    # combine subresults
    A = [0] * len(aX)
    Nthird = len(aX) // 3
    omega_Nthird = (omega**Nthird) % Q
    point = 1
    for i in range(Nthird):
        
        x = point
        xx = (x * x) % Q
        A[i                  ] = (B[i] + x * C[i] + xx * D[i]) % Q
        
        x = x * omega_Nthird % Q
        xx = (x * x) % Q
        A[i + Nthird         ] = (B[i] + x * C[i] + xx * D[i]) % Q
        
        x = x * omega_Nthird % Q
        xx = (x * x) % Q
        A[i + Nthird + Nthird] = (B[i] + x * C[i] + xx * D[i]) % Q

        point = (point * omega) % Q
        
    return A

def fft3_backward(A):
    N_inv = inverse(len(A))
    return [ (a * N_inv) % Q for a in fft3_forward(A, inverse(OMEGA3)) ]

coefs = [1,2,3,4,5,6,7,8,9]
values = fft3_forward(coefs)
coefs_recovered = fft3_backward(values)
assert(coefs == coefs_recovered)

In [ ]:
(198 * OMEGA3) % Q

In [ ]:
1+198+234

In [ ]:
150+256+27

# Parameter generation

In [ ]:
N = 8
T = 4
K = 3
Q = 433 # 6 * (T+K+1) * (N+1) + 1

G = 

OMEGA2 = 354
assert((OMEGA2**(T+K+1)) % Q == 1)

OMEGA3 = 150
assert((OMEGA3**(N+1)) % Q == 1)

In [ ]:
set( (OMEGA2 ** e) % Q for e in range(Q) )

In [ ]:
set( (OMEGA3 ** e) % Q for e in range(Q) )

In [ ]:
2*2*2*2*3*3*3 + 1

In [ ]:
import math
from functools import reduce

#
# naive algorithms for primality testing and factoring -- but sufficient for these purposes
#

def is_prime(n):
    if n % 2 == 0 and n > 2: 
        return False
    for i in range(3, math.floor(math.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True

def factor(n):
    return set(reduce(list.__add__, 
                ([i, n//i] for i in range(2, math.floor(math.sqrt(n))+1) if n % i == 0)))

In [ ]:
def find_field(min_size, n, m):
    
    offset = math.ceil( (min_size-1) / (n*m) )
    for k in range(offset, offset+1000):
        
        # candidate prime
        p = k * n * m + 1
        
        # make sure p is above lower bound and prime
        if not p >= min_size: continue
        if not is_prime(p): continue
        
        # find generator of Z_p
        
        factors = factor(p-1)
        for g in range(2, p):
            for q in factors:
                e = (p-1) // q
                if ((g**e) % p) == 1:
                    break
            else:
                assert((g**(p-1)) % p == 1)
                return (p, g)

In [ ]:
n = 8
m = 9
(p, g) = find_field(200, n, m)

print(p, g)

In [ ]:
omega_n = g**( (p-1) // n ) % p
omega_m = g**( (p-1) // m ) % p

group_n = [ omega_n**e % p for e in range(0, n) ]
group_m = [ omega_m**e % p for e in range(0, m) ]
assert(set(group_n).intersection(set(group_m)) == set([1]))

print(omega_n, omega_m)

In [ ]:

set( (OMEGA2 ** e) % Q for e in range(Q) )

In [ ]:
q = 18446744073710162953
p = q - 1
p = p // 2**3
p = p // 3**4
p = p // 7**1
p = p // 17**1
p = p // 332509**1
p = p // 719439619**1

In [ ]:
q == (2**3) * (3**4) * (7) * (17) * (332509) * (719439619) + 1

In [ ]:


found factor 2
found factor 3
found factor 7
found factor 17
found factor 332509
found factor 719439619
Trying large primes
Generator is 5
8-th root is 18296739951669628889
9-th root is 6042937017363401072

# Dump

In [ ]:
import random

def miller_rabin(n, k=10):
    if n == 2:
        return True
    if not n & 1:
        return False

    def check(a, s, d, n):
        x = pow(a, d, n)
        if x == 1:
            return True
        for i in range(s - 1):
            if x == n - 1:
                return True
            x = pow(x, 2, n)
        return x == n - 1

    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1

    for i in range(k):
        a = random.randrange(2, n - 1)
        if not check(a, s, d, n):
            return False
        
    return True

In [ ]:
miller_rabin(433)